In [1]:
# A mashed-together python file

import matplotlib.pyplot as plt
import numpy as np
import torch; torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import numpy as np
import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 200

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

dataList = []
ytargetList = []
# load first 10 datapoints for testing
# x: (n_samples, x_ver_dim, x_hor_dim) with x_ver_dim = x_hor_dim = 512 (so in total 512^2 pixel)
# y: (n_samples, y_dim) with y_dim=2 (the julia constant and the starting coordinate)

dataList = []
ytargetList = []
# load first 10 datapoints for testing
for fileIndex in range(2000):
    with open('../trainingData/data'+str(fileIndex)+'.jset', 'r') as file:
        ytargetString = file.readline()[2:].rstrip()
        lines = file.readlines()
    ytargetList.append(np.fromstring(ytargetString, dtype=float, sep=','))
    #dataList.append([np.fromstring(line, dtype=float, sep=',') for line in lines])
    x = [np.fromstring(line, dtype=float, sep=',') for line in lines]
    x = np.array(x)
    for i in range(26):
        for j in range(26):
            x[i, j] = np.mean(x[i*20:(i+1)*20, j*20:(j+1)*20])
    x = x[:26,:26].reshape(676)
    #x = x.reshape(271441)
    dataList.append(x)
#convert lists into numpy matrices
data = np.array(dataList)
ytarget = np.array(ytargetList)
print('data points:', data.shape)
print('targets:', ytarget.shape)

class Encoder(nn.Module):
    def __init__(self, latent_dims):
        super(Encoder, self).__init__()
        self.linear1 = nn.Linear(676, 512)
        self.linear2 = nn.Linear(512, latent_dims)
    
    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.linear1(x))
        return self.linear2(x)
        
class Decoder(nn.Module):
    def __init__(self, latent_dims):
        super(Decoder, self).__init__()
        self.linear1 = nn.Linear(latent_dims, 512)
        self.linear2 = nn.Linear(512, 676)
        
    def forward(self, z):
        z = F.relu(self.linear1(z))
        z = torch.sigmoid(self.linear2(z))
        return z.reshape((-1, 1, 26, 26))

class Autoencoder(nn.Module):
    def __init__(self, latent_dims):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(latent_dims)
        self.decoder = Decoder(latent_dims)
    
    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

def train(autoencoder, data, epochs=20):
    opt = torch.optim.Adam(autoencoder.parameters())
    for epoch in range(epochs):
        for x, _ in data:
            x = x.to(device) # GPU
            opt.zero_grad()
            x_hat = autoencoder(x)
            loss = ((x - x_hat)**2).sum()
            loss.backward()
            opt.step()
    return autoencoder

latent_dims = 2
autoencoder = Autoencoder(latent_dims).to(device) # Initialize an autoencoder and send it to GPU

data = data.reshape((-1, 1, 26, 26)) # reshape data -1, 1 indicate the direction

tensor_data = torch.Tensor(data) # transform to torch tensor
tensor_y = torch.Tensor(ytarget)

dataset = torch.utils.data.TensorDataset(tensor_data,tensor_y) # create dataset
new_data = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True) # create dataloader

autoencoder = train(autoencoder, new_data)

cuda
data points: (2000, 676)
targets: (2000, 2)


In [2]:
#plt.imshow(data[0, 0])
#x = torch.Tensor(data[0, 0]).to(device)
#x_hat = autoencoder.forward(x)
x =  new_data[0,0]
#print(x.shape)
#plt.imshow(x[55][0])
x = x.reshape((1, 26,26))
print(x.shape)
output = autoencoder.decoder(autoencoder.encoder(x))
img = output.to('cpu').detach().numpy()
#print(output)
plt.imshow(img[0][0])
#plt.imshow()

"""
def plot_reconstructed(autoencoder, r0=(-5, 10), r1=(-10, 5), n=12):
    w = 26
    img = np.zeros((n*w, n*w))
    for i, y in enumerate(np.linspace(*r1, n)):
        for j, x in enumerate(np.linspace(*r0, n)):
            z = torch.Tensor([[x, y]]).to(device)
            x_hat = autoencoder.decoder(z)
            x_hat = x_hat.reshape(26, 26).to('cpu').detach().numpy()
            img[(n-1-i)*w:(n-1-i+1)*w, j*w:(j+1)*w] = x_hat
    plt.imshow(img, extent=[*r0, *r1])
"""
#plot_reconstructed(autoencoder)

TypeError: 'DataLoader' object is not subscriptable